In [1]:
from __future__ import annotations 

import pathlib

import xarray as xr
import numpy as np
import pandas as pd

In [2]:
BASE = pathlib.Path("/home/panos/jeodpp.jrc.ec.europa.eu/jeodpp.jrc.ec.europa.eu/ftp/private/zyWUYa4wk/nxGGjsuRHYrOuTx1/output-ftp/mavropa/post/")
SIM = pathlib.Path("/home/panos/jeodpp.jrc.ec.europa.eu/jeodpp.jrc.ec.europa.eu/ftp/private/zyWUYa4wk/nxGGjsuRHYrOuTx1/output-ftp/mavropa/post/sim/")
OBS = pathlib.Path("/home/panos/jeodpp.jrc.ec.europa.eu/jeodpp.jrc.ec.europa.eu/ftp/private/zyWUYa4wk/nxGGjsuRHYrOuTx1/output-ftp/mavropa/post/obs/")

In [3]:
sims = sorted(SIM.glob("*.csv"))
obss = sorted(OBS.glob("*.csv"))

In [4]:
s = sims[0]
s

PosixPath('/home/panos/jeodpp.jrc.ec.europa.eu/jeodpp.jrc.ec.europa.eu/ftp/private/zyWUYa4wk/nxGGjsuRHYrOuTx1/output-ftp/mavropa/post/sim/sim_Denmark-Hirtshals.csv')

In [5]:
df = pd.read_csv(s)

In [6]:
df

,Unnamed: 0,time,elev
0,l0,2018-01-01 00:06:40,-0.001119
1,l0,2018-01-01 00:13:20,-0.002908
2,l0,2018-01-01 00:20:00,-0.005010
3,l0,2018-01-01 00:26:40,-0.007666
4,l0,2018-01-01 00:33:20,-0.011510
...,...,...,...
1226011,l5,2020-07-22 23:33:20,0.063473
1226012,l5,2020-07-22 23:40:00,0.061961
1226013,l5,2020-07-22 23:46:40,0.060151
1226014,l5,2020-07-22 23:53:20,0.058085


In [26]:
locations = pd.read_csv(BASE / "locations.csv")
locations

,Unnamed: 0,ID,Name,Group,Latency,Details,Last Date,Max. Time,Min. Time,lat,lon,Status
0,47,894,Denmark-Hirtshals,TD UNESCO,1h 1m,Details,09 Apr 2018 10:40,09 Apr 2018 10:40,31 Dec 2009 00:10,57.600,9.970,Active
1,48,1692,"Denmark-Ittoqqortoormiit,Scoresbysund",TD UNESCO,46m,Details,09 Apr 2018 10:54,09 Apr 2018 10:54,02 May 2011 22:27,70.484,-21.962,Active
2,49,1877,Dunkerque,TD UNESCO,57m,Details,09 Apr 2018 10:43,09 Apr 2018 10:43,17 Jul 2012 02:12,51.048,2.367,Active
3,50,1438,France-Ajaccio,TD UNESCO,55m,Details,09 Apr 2018 10:45,09 Apr 2018 10:45,29 Apr 2011 19:08,41.920,8.760,Active
4,51,1599,France-Ajaccio2,TD UNESCO,1h 1m,Details,09 Apr 2018 10:40,09 Apr 2018 10:40,29 Apr 2011 19:07,41.920,8.760,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
180,227,1940,UK-WestBayHarbour,TD UNESCO,31m,Details,09 Apr 2018 11:09,09 Apr 2018 09:11,18 Oct 2012 17:10,50.710,-2.764,Active
181,228,1941,UK-Weymouth,TD UNESCO,1h 11m,Details,09 Apr 2018 10:30,09 Apr 2018 08:45,18 Oct 2012 17:30,50.610,-2.450,Active
182,229,1942,UK-Whitby,TD UNESCO,1h 11m,Details,09 Apr 2018 10:30,09 Apr 2018 08:45,18 Oct 2012 17:30,54.490,-0.610,Active
183,230,1943,UK-Wic,TD UNESCO,1h 11m,Details,09 Apr 2018 10:30,09 Apr 2018 08:45,18 Oct 2012 17:30,58.440,-3.090,Active


In [157]:
def get_da_from_csv(csv: pathlib.Path) -> xr.DataArray:
    df = pd.read_csv(csv, names=["level", "time", "elevation"], header=0)
    df = df.drop_duplicates().reset_index(drop=True)
    loc = locations[locations.Name==csv.stem.split("_")[1]]
    df = df.assign(
        lat=loc.lat,
        lon=loc.lon,
    )
    df = df.set_index(["lat", "lon", "time", "level"])
    da = df.to_xarray()
    return da

In [169]:
da = get_da_from_csv(csv)

In [174]:
da

<xarray.Dataset>
Dimensions:    (lat: 1, lon: 1, time: 201744, level: 6)
Coordinates:
  * lat        (lat) float64 57.6
  * lon        (lon) float64 9.97
  * time       (time) object '2018-01-01 00:06:40' ... '2020-07-23 00:00:00'
  * level      (level) object 'l0' 'l1' 'l2' 'l3' 'l4' 'l5'
Data variables:
    elevation  (lat, lon, time, level) float64 -0.001119 nan nan ... nan 0.05581

In [158]:
%%time 
csvs = (list(SIM.glob(f"sim_{row.Name}.csv"))[0] for row in locations.itertuples())
das = [get_da_from_csv(csv) for csv in list(csvs)[:2]]

In [159]:
das[0]

<xarray.Dataset>
Dimensions:    (lat: 1, lon: 1, time: 201744, level: 6)
Coordinates:
  * lat        (lat) float64 57.6
  * lon        (lon) float64 9.97
  * time       (time) object '2018-01-01 00:06:40' ... '2020-07-23 00:00:00'
  * level      (level) object 'l0' 'l1' 'l2' 'l3' 'l4' 'l5'
Data variables:
    elevation  (lat, lon, time, level) float64 -0.001119 nan nan ... nan 0.05581

In [160]:
%%time 
ds = xr.concat(
    das,
    dim=["lat", "lon", "time", "level"],
)

ValueError: conflicting sizes for dimension 'concat_dim': length 4 on 'concat_dim' and length 2 on {'lat': 'lat', 'lon': 'lon', 'time': 'time', 'level': 'level', 'concat_dim': 'elevation'}

In [155]:
csv.stem.split("_")[1]

'Denmark-Hirtshals'

In [56]:
ds = xr.Dataset(
    coords=dict(
        #station=locations.ID,
        levels=[f"l{i}" for i in range(6)],
    ),
    data_vars=dict(
        lat=(("station"), locations.lat),
        lon=(("station"), locations.lon),
        station=(("station"), locations.ID),
    )
)
ds
             

<xarray.Dataset>
Dimensions:  (station: 185, levels: 6)
Coordinates:
  * station  (station) int64 894 1692 1877 1438 1599 ... 1941 1942 1943 1944
  * levels   (levels) <U2 'l0' 'l1' 'l2' 'l3' 'l4' 'l5'
Data variables:
    lat      (station) float64 57.6 70.48 51.05 41.92 ... 54.49 58.44 54.65
    lon      (station) float64 9.97 -21.96 2.367 8.76 ... -0.61 -3.09 -3.57

<xarray.Dataset>
Dimensions:    (level: 6, time: 201744)
Coordinates:
  * level      (level) object 'l0' 'l1' 'l2' 'l3' 'l4' 'l5'
  * time       (time) object '2018-01-01 00:06:40' ... '2020-07-23 00:00:00'
Data variables:
    elevation  (level, time) float64 -0.001119 -0.002908 ... 0.05809 0.05581

In [108]:
da = xr.DataArray(
    data=df.elevation,
    coords=[df.level, df.time],
)

ValueError: coords is not dict-like, but it has 2 items, which does not match the 1 dimensions of the data

In [73]:
df.set_index(["time", "level"]).index.nunique()#.to_xarray()

1207224

In [81]:
aa = df.set_index(["time", "level"])
aa[aa.duplicated()]

,,elevation
time,level,
2018-01-02 05:46:40,l2,0.599808
2018-01-01 23:53:20,l0,0.295388
2018-01-02 16:33:20,l1,0.421142
2018-01-03 23:00:00,l3,0.638955
2018-01-04 04:33:20,l4,0.596267
...,...,...
2020-07-22 20:26:40,l5,0.025446
2020-07-22 21:13:20,l5,0.046286
2020-07-22 21:26:40,l5,0.051218


In [87]:
df[(df.level=="l2") & (df.time=="2018-01-02 05:46:40")]

,level,time,elevation
267,l2,2018-01-02 05:46:40,0.599808


In [90]:
df[df.duplicated()]

,level,time,elevation
1151496,l0,2020-05-24 00:06:40,0.471216
1151497,l0,2020-05-24 00:13:20,0.467507
1151498,l0,2020-05-24 00:20:00,0.463753
1151499,l0,2020-05-24 00:26:40,0.459978
1151500,l0,2020-05-24 00:33:20,0.456177
...,...,...,...
1170283,l5,2020-06-09 23:33:20,-0.095404
1170284,l5,2020-06-09 23:40:00,-0.094611
1170285,l5,2020-06-09 23:46:40,-0.093685
1170286,l5,2020-06-09 23:53:20,-0.092624


In [91]:
df[(df.level=="l0") & (df.time=="2020-05-24 00:06:40")]

,level,time,elevation
1132704,l0,2020-05-24 00:06:40,0.471216
1151496,l0,2020-05-24 00:06:40,0.471216


In [97]:
print(len(df))
print(len(df.drop_duplicates()))
len(df) - len(df.drop_duplicates())

1226016
1207224


18792

In [99]:
dups = df[df.duplicated()]
dups

,level,time,elevation
1151496,l0,2020-05-24 00:06:40,0.471216
1151497,l0,2020-05-24 00:13:20,0.467507
1151498,l0,2020-05-24 00:20:00,0.463753
1151499,l0,2020-05-24 00:26:40,0.459978
1151500,l0,2020-05-24 00:33:20,0.456177
...,...,...,...
1170283,l5,2020-06-09 23:33:20,-0.095404
1170284,l5,2020-06-09 23:40:00,-0.094611
1170285,l5,2020-06-09 23:46:40,-0.093685
1170286,l5,2020-06-09 23:53:20,-0.092624


In [107]:
df[df.time.isin(dups.time) & df.elevation.isin(dups.elevation)].sort_values(["level", "time"])

,level,time,elevation
1132704,l0,2020-05-24 00:06:40,0.471216
1151496,l0,2020-05-24 00:06:40,0.471216
1132705,l0,2020-05-24 00:13:20,0.467507
1151497,l0,2020-05-24 00:13:20,0.467507
1132706,l0,2020-05-24 00:20:00,0.463753
...,...,...,...
1170285,l5,2020-06-09 23:46:40,-0.093685
1151494,l5,2020-06-09 23:53:20,-0.092624
1170286,l5,2020-06-09 23:53:20,-0.092624
1151495,l5,2020-06-10 00:00:00,-0.091447


In [101]:
df[df.time=="2020-06-09 00:06:40"]

,level,time,elevation
1150740,l5,2020-06-09 00:06:40,-0.040786
1151280,l4,2020-06-09 00:06:40,-0.074496
1169532,l5,2020-06-09 00:06:40,-0.040786
1170072,l4,2020-06-09 00:06:40,-0.074496
1170612,l3,2020-06-09 00:06:40,-0.065008
1171152,l2,2020-06-09 00:06:40,-0.078199
1171692,l1,2020-06-09 00:06:40,-0.091944
1172232,l0,2020-06-09 00:06:40,-0.091944
